In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
from tensorflow import keras
# Import necessary tools
from keras.src.saving import custom_object_scope
# Import the actual DepthwiseConv2D class to wrap it
from keras.src.layers import DepthwiseConv2D

# ----------------------------------------------------
# 1. Model Loading and Conversion (with error fix)
# ----------------------------------------------------

# Define a function to create a new class/constructor that intercepts the
# layer's configuration and removes the problematic 'groups' parameter.
def DepthwiseConv2D_Fixed(*args, **kwargs):
    """A wrapper for DepthwiseConv2D to fix the 'groups' compatibility issue."""
    # The actual configuration is typically nested in the first positional argument 
    # when an H5 config is being loaded.
    if args and isinstance(args[0], dict):
        config = args[0].get('config', {})
        # Remove the 'groups' key from the actual config dictionary
        if 'groups' in config:
            del config['groups']
            args[0]['config'] = config
    
    # Also check if 'groups' is accidentally passed as a direct keyword argument
    if 'groups' in kwargs:
        del kwargs['groups']
    
    # Return the instance of the original class with the modified arguments
    return DepthwiseConv2D(*args, **kwargs)


# Load the old h5 model using a custom object scope
try:
    # Use the corrected wrapper class in the custom_object_scope
    with custom_object_scope({'DepthwiseConv2D': DepthwiseConv2D_Fixed}):
        model = keras.models.load_model("Model/keras_model.h5", compile=False)
        print("INFO: Model loaded successfully using compatibility wrapper.")
except Exception as e:
    print(f"FATAL ERROR: Could not load the model 'Model/keras_model.h5'.")
    print(f"Details: {e}")
    # Define model as None so the NameError is avoided later
    model = None 

# If the model failed to load, stop the program to avoid NameError
if model is None:
    # This also solves your second error: NameError: name 'model' is not defined
    exit()
    
# Save the model as the new Keras 3 format (.keras) for future compatibility
model.save("Model/keras_model.keras")

# ----------------------------------------------------
# 2. Initialization
# ----------------------------------------------------

cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
# Use the newly saved, compatible .keras file
classifier = Classifier("Model/keras_model.keras", "Model/labels.txt")

# ... rest of your code (which is unchanged) ...

# Setup parameters
offset = 20
imgSize = 300
folder = "data/C" 
counter = 0        
labels = ["A", "B", "C"] 

# ----------------------------------------------------
# 3. Main Loop
# ----------------------------------------------------

while True:
    success, img = cap.read()
    if not success:
        print("Failed to read from webcam.")
        break
        
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        
        # Create a square white canvas
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        
        # Crop the hand image with offset
        imgCrop = img[max(0, y - offset):min(img.shape[0], y + h + offset), 
                      max(0, x - offset):min(img.shape[1], x + w + offset)]
        
        # Check if the crop is valid (not empty)
        if imgCrop.size == 0:
            continue

        h_crop, w_crop, _ = imgCrop.shape
        aspectRatio = h_crop / w_crop

        # Resize the cropped image and place it on the white canvas
        if aspectRatio > 1: # Height is greater than width
            k = imgSize / h_crop
            wCal = math.ceil(k * w_crop)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
            
        else: # Width is greater than or equal to height
            k = imgSize / w_crop
            hCal = math.ceil(k * h_crop)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
            
        # Get the prediction
        prediction, index = classifier.getPrediction(imgWhite, draw=False)
        
        # -------------------
        # Drawing on imgOutput
        # -------------------
        
        # 1. Prediction background box
        cv2.rectangle(imgOutput, (x - offset, y - offset - 50),
                      (x - offset + 90, y - offset), (255, 0, 255), cv2.FILLED)
                      
        # 2. Prediction text
        cv2.putText(imgOutput, labels[index], (x, y - 26), 
                    cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
                    
        # 3. Hand bounding box
        cv2.rectangle(imgOutput, (x - offset, y - offset),
                      (x + w + offset, y + h + offset), (255, 0, 255), 4)
                      
        # -------------------
        # Display cropped images for debugging/visualization
        # -------------------
        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)
        
    # Display the final output image
    cv2.imshow("Image", imgOutput)
    
    # Wait for a key press and exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

C:\Users\palak\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


FATAL ERROR: Could not load the model 'Model/keras_model.h5'.
Details: Exception encountered when calling Sequential.call().

Layer "functional_4" expects 1 input(s), but it received 2 input tensors. Inputs received: [<KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, ragged=False, name=keras_tensor_163>, <KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, ragged=False, name=keras_tensor_164>]

Arguments received by Sequential.call():
  • args=(('<KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, ragged=False, name=keras_tensor_163>', '<KerasTensor shape=(None, 7, 7, 1280), dtype=float32, sparse=False, ragged=False, name=keras_tensor_164>'),)
  • kwargs={'mask': ('None', 'None')}


AttributeError: 'NoneType' object has no attribute 'save'